In [1]:
import pandas as pd
import tqdm
import logging
from socket import timeout
import queue
import threading
import urllib
import shutil

In [2]:
df = pd.read_csv("../datas/raw/full_omdb.csv" , index_col=0, encoding="ISO-8859-1")

In [3]:
df = df[["imdbID", "Poster"]].set_index("imdbID").dropna(axis=0)

In [4]:
logger = logging.getLogger('poster')
hdlr = logging.FileHandler('poster.log')
logger.addHandler(hdlr) 
logger.setLevel(logging.WARNING)

In [5]:
def extract_content():
    while True:
        row = q.get()
        if row is None:
            break
        id = row[0]
        path = row[1].values[0]
        output_file = "../datas/posters/"+id+".jpg"
        try:
            with urllib.request.urlopen(path, timeout = 10) as response, open(output_file, 'wb') as out_file:
                shutil.copyfileobj(response, out_file)
        except (urllib.error.HTTPError, urllib.error.URLError) as error:
            logger.error('Data of %s not retrieved because %s (%s)', id, error, path)
        except timeout:
            logger.error('socket timed out - Key %s', id)
        q.task_done()

In [6]:
q = queue.Queue()
num_worker_threads = 10

print("launch thread")
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=extract_content)
    t.start()
    threads.append(t)
    
print("feeding queue")
for row in df.iterrows():
    q.put(row)
q.join()

for i in range(num_worker_threads):
    q.put(None)

for t in threads:
    t.join()
print("end")

launch thread
feeding queue
end
